# TFGENZOO_Tutorial (2) How to add conditional Input into the invertible layer

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MokkeMeguru/TFGENZOO/blob/master/tutorials/01_What_is_the_invertible_layer.ipynb)

Some novels of Flow-based Model requires Conditional Input to control some attribution e.g. image style.

Below image is the example from the paper,
"Guided Image Generation 
with Conditional Invertible Neural Networks (cINN)" 


![cinn.png](https://raw.githubusercontent.com/MokkeMeguru/TFGENZOO/master/tutorials/img/cinn.png)


# Conditional Input into the AffineCoupling Layer

Conditional Input usually concated in the AffineCoupling Layer.

In [1]:
# install TFGENZOO
!pip install TFGENZOO==1.2.4.post7

In [2]:
import tensorflow as tf
from TFGENZOO.flows.cond_affine_coupling import ConditionalAffineCoupling

## Conditional Affine Coupling forward formulation:

```
[x_1, x_2] = split(x)
log_scale, shift <- NN([x_1, cond])
z_1 = x_1
z_2 = (x_2 + shift) * scale
z = concat([z_1, z_2])

log_det_jacobian = sum(log(scale))
```

## NN layer to deal with Conditional Input
Here is the basic NN Layer proposed in cINN.

It is same as TFGENZOO.layers.resnet.ShallowResNet.

In [3]:
from TFGENZOO.flows.utils import Conv2D, Conv2DZeros

def ShallowResNet(
    inputs: tf.keras.Input,
    cond: tf.keras.Input = None,
    width: int = 512,
    out_scale: int = 2,
):
    """ResNet of OpenAI's Glow
    Args:
        inputs (tf.Tensor): input tensor rank == 4
        cond   (tf.Tensor): input tensor rank == 4 (optional)
        width        (int): hidden width
        out_scale    (int): output channel width scale
    Returns:
        model: tf.keras.Model
    """
    _inputs = inputs if cond is None else tf.concat([inputs, cond], axis=-1)

    conv1 = Conv2D(width=width)
    conv2 = Conv2D(width=width)
    conv_out = Conv2DZeros(width=int(inputs.shape[-1] * out_scale))

    outputs = _inputs
    outputs = tf.keras.layers.ReLU()(conv1(outputs))
    outputs = tf.keras.layers.ReLU()(conv2(outputs))
    outputs = conv_out(outputs)
    return tf.keras.Model(inputs if cond is None else [inputs, cond], outputs)

# Conditional AffineCouplig Layer 

To construct Conditional AffineCoupling Layer, we can take two type.


## scale_shift_net_template with tf.keras.layer.build (recommend)

This solution uses the template function which generates tf.keras.Model


In [4]:
from TFGENZOO.layers.resnet import ShallowResNet
from TFGENZOO.flows.cond_affine_coupling import ConditionalAffineCoupling

# You need to define the shape of condional input
cond = tf.keras.Input([16, 16, 32])

condAffineCoupling = ConditionalAffineCoupling(
    scale_shift_net_template=lambda x: ShallowResNet(x, cond))

# To build the template, we need to build your model.
condAffineCoupling.build([None, 16, 16, 12])

x = tf.random.normal([32, 16, 16, 12])
c = tf.random.normal([32, 16, 16, 32])

# here is the forward function
z, ldj = condAffineCoupling(x, cond=c)

# here is the inverse function
rev_x, ildj = condAffineCoupling(z, cond=c, inverse=True)

print(f'reconstruction loss is {tf.reduce_mean((rev_x - x)**2)}, and also, reconstruction effect is  {tf.reduce_mean(ldj + ildj)}')

initialization at actnorm_activation
initialization at actnorm_activation_1
reconstruction loss is 0.0, and also, reconstruction effect is  0.0


## scale_shift_net with tf.keras.Model

This solution uses the tf.keras.Model directly. We need to find whole layer's dimention.



In [5]:
# You need to define the shape of condional input
cond = tf.keras.Input([16, 16, 32])
x_temp = tf.keras.Input([16, 16, 12 // 2])

condAffineCoupling = ConditionalAffineCoupling(
    scale_shift_net=ShallowResNet(x_temp, cond))

# To build the template, we need to build your model.
condAffineCoupling.build([None, 16, 16, 12])

x = tf.random.normal([32, 16, 16, 12])
c = tf.random.normal([32, 16, 16, 32])

# here is the forward function
z, ldj = condAffineCoupling(x, cond=c)

# here is the inverse function
rev_x, ildj = condAffineCoupling(z, cond=c, inverse=True)

print(f'reconstruction loss is {tf.reduce_mean((rev_x - x)**2)}, and also, reconstruction effect is  {tf.reduce_mean(ldj + ildj)}')

initialization at actnorm_activation_2
initialization at actnorm_activation_3
reconstruction loss is 0.0, and also, reconstruction effect is  0.0


# Conditional Flow Module

In previous tutorial, we introduce the TFGENZOO.flows.FlowModule to construt a step of flow.

However,  in now, we need to input the condtional input into FlowModule.

To tackle this problem, we serve TFGENZOO.flows.flowbase.ConditionalFlowModule.

In [6]:
from TFGENZOO.flows.flowbase import ConditionalFlowModule
from TFGENZOO.flows import Inv1x1Conv, Actnorm

channel = 12
assert channel % 2 == 0

cond = tf.keras.Input([16, 16, 32])

fm = []
fm.append(Actnorm())
fm.append(Inv1x1Conv())
cac = ConditionalAffineCoupling(scale_shift_net_template=lambda x: ShallowResNet(x, cond=cond))
fm.append(cac)

stepOfFlow= ConditionalFlowModule(fm)

stepOfFlow.build([None, 16, 16, channel])


x = tf.random.normal([32, 16, 16, channel])
c = tf.random.normal([32, 16, 16, 32])

# here is the forward function
z, ldj = stepOfFlow(x, cond=c)

# here is the inverse function
rev_x, ildj = stepOfFlow(z, cond=c, inverse=True)

print(f'reconstruction loss is {tf.reduce_mean((rev_x - x)**2)}, and also, reconstruction effect is  {tf.reduce_mean(ldj + ildj)}')

initialization at actnorm
initialization at actnorm_activation_4
initialization at actnorm_activation_5
reconstruction loss is 3.656933629070207e-14, and also, reconstruction effect is  0.0


# Conclusion
- To add the Condtional Input into Flow-based Model, we usually modify Coupling Layer.
- TFGENZOO supports Conditional Coupling by ConditionalAffineCoupling and ConditionalFlowModule